In [ ]:
!pip install pyngrok
%cd /content/
!git clone https://github.com/facebookresearch/av_hubert.git

%cd av_hubert
!git submodule init
!git submodule update
!pip install scipy
!pip install sentencepiece
!pip install python_speech_features
!pip install scikit-video

%cd fairseq
!pip install ./
!pip install fer
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content
fatal: destination path 'av_hubert' already exists and is not an empty directory.
/content/av_hubert
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/av_hubert/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/av_hubert/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-1.0.0a0

In [ ]:
!mkdir -p /content/data/misc/
!mkdir -p /content/data/framedata
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!wget --content-disposition https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy -O /content/data/misc/20words_mean_face.npy
!wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O /content/data/finetune-model.pt

--2023-06-02 08:12:31--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’

/content/data/misc/ 100%[===================>]  61.07M  15.7MB/s    in 6.0s    

2023-06-02 08:12:37 (10.3 MB/s) - ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bzip2: Output file /content/data/misc/shape_predictor_68_face_landmarks.dat already exists.
--2023-06-02 08:12:37--  https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import os
!killall ngrok
pid = os.getpid()
!kill -9 $pid

In [ ]:
%cd /content/av_hubert/avhubert/
import dlib, cv2, os
import numpy as np
import skvideo
import skvideo.io
from tqdm import tqdm
from preparation.align_mouth import landmarks_interpolate, crop_patch, write_video_ffmpeg
from IPython.display import HTML
from base64 import b64encode

import tempfile
from argparse import Namespace
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.configs import GenerationConfig
from IPython.display import HTML

import threading
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from pyngrok import ngrok

from fer import FER


def play_video(video_path, width=200):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML(f"""
  <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
  </video>
  """)

def detect_landmark(image, detector, predictor):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = detector(gray, 1)
    coords = None
    for (_, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        coords = np.zeros((68, 2), dtype=np.int32)
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

def preprocess_video(input_video_path, output_video_path, face_predictor_path, mean_face_path):
  print(1)
  detector = dlib.get_frontal_face_detector()
  print(1)
  predictor = dlib.shape_predictor(face_predictor_path)
  print(1)
  STD_SIZE = (256, 256)
  print(1)
  mean_face_landmarks = np.load(mean_face_path)
  print(1)
  stablePntsIDs = [33, 36, 39, 42, 45]
  print(1)
  videogen = skvideo.io.vread(input_video_path)
  print(1)
  frames = np.array([frame for frame in videogen])
  print(1)
  landmarks = []
  for frame in tqdm(frames):
      landmark = detect_landmark(frame, detector, predictor)
      landmarks.append(landmark)
  preprocessed_landmarks = landmarks_interpolate(landmarks)
  rois = crop_patch(input_video_path, preprocessed_landmarks, mean_face_landmarks, stablePntsIDs, STD_SIZE, 
                        window_margin=12, start_idx=48, stop_idx=68, crop_height=96, crop_width=96)
  write_video_ffmpeg(rois, output_video_path, "/usr/bin/ffmpeg")
  return

def getEmotion():
  folder_name = '/content/data/framedata'
  map_of_emotions = dict();
  map_of_emotions[0] = 'angry';
  map_of_emotions[1] = 'cheerful';
  map_of_emotions[2] = 'fearful'; 
  map_of_emotions[3] = 'sad';
  map_of_emotions[4] = 'chat';
  map_of_emotions[5] = 'disgruntled';
  map_of_emotions[6] = 'excited';
  list_of_emotions=[];
  for i in range(0,7):
      list_of_emotions.append(0)
  item = '/content/data/clip.mp4'
  vid = cv2.VideoCapture(item)
  currentframe = 0
  emo_detector = FER(mtcnn=True)
  while (True):
      success, frame = vid.read()
      if success:
          name1 = folder_name+'/frame' + str(currentframe) + '.jpg'
          cv2.imwrite(name1, frame)
          currentframe += 1
          test_image_one = cv2.imread(name1)
          dominant_emotion,score = emo_detector.top_emotion(test_image_one)
          if dominant_emotion == "angry":
              list_of_emotions[0]+=1; 
          elif dominant_emotion == "happy":
              list_of_emotions[1]+=1;
          elif dominant_emotion == "fear":
              list_of_emotions[2]+=1;
          elif dominant_emotion == "sad":
              list_of_emotions[3]+=1;
          elif dominant_emotion == "neutral":
              list_of_emotions[4]+=1;
          elif dominant_emotion == "disgust":
              list_of_emotions[5]+=1;
          elif dominant_emotion == "surprise":
              list_of_emotions[6]+=1;
      else:
          break
  vid.release()
  cv2.destroyAllWindows()
  max_emotion_count = list_of_emotions[0]
  max_emotion = map_of_emotions[0]
  for i in range(1,7):
      count = list_of_emotions[i]
      if count>max_emotion_count:
          max_emotion_count = count
          max_emotion = map_of_emotions[i]
  #print('MAXIMUM EMOTION  - ', max_emotion)
  return max_emotion

def predict(video_path, ckpt_path, user_dir):
  print(0)
  num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
  print(1)
  data_dir = tempfile.mkdtemp()
  print(2)
  tsv_cont = ["/\n", f"test-0\t{video_path}\t{None}\t{num_frames}\t{int(16_000*num_frames/25)}\n"]
  print(3)
  label_cont = ["DUMMY\n"]
  print(4)
  with open(f"{data_dir}/test.tsv", "w") as fo:
    fo.write("".join(tsv_cont))
  with open(f"{data_dir}/test.wrd", "w") as fo:
    fo.write("".join(label_cont))
  print(5)
  utils.import_user_module(Namespace(user_dir=user_dir))
  print(6)
  modalities = ["video"]
  gen_subset = "test"
  gen_cfg = GenerationConfig(beam=20)
  print(7)
  models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
  print(8)
  models = [model.eval().cuda() for model in models]
  print(9)
  saved_cfg.task.modalities = modalities
  saved_cfg.task.data = data_dir
  saved_cfg.task.label_dir = data_dir
  print(10)
  task = tasks.setup_task(saved_cfg.task)
  task.load_dataset(gen_subset, task_cfg=saved_cfg.task)
  print(11)
  generator = task.build_generator(models, gen_cfg)
  print(12)

  def decode_fn(x):
      dictionary = task.target_dictionary
      symbols_ignore = generator.symbols_to_strip_from_output
      symbols_ignore.add(dictionary.pad())
      return task.datasets[gen_subset].label_processors[0].decode(x, symbols_ignore)
  
  itr = task.get_batch_iterator(dataset=task.dataset(gen_subset)).next_epoch_itr(shuffle=False)
  sample = next(itr)
  sample = utils.move_to_cuda(sample)
  hypos = task.inference_step(generator, models, sample)
  ref = decode_fn(sample['target'][0].int().cpu())
  hypo = hypos[0][0]['tokens'].int().cpu()
  hypo = decode_fn(hypo)
  return hypo



face_predictor_path = "/content/data/misc/shape_predictor_68_face_landmarks.dat"
mean_face_path = "/content/data/misc/20words_mean_face.npy"
origin_clip_path = "/content/data/clip.mp4"
mouth_roi_path = "/content/data/roi.mp4"


ckpt_path = "/content/data/finetune-model.pt"
ckpt_path_hindi = "/content/gdrive/MyDrive/final/exp1/checkpoints/checkpoint_last.pt"
user_dir = "/content/av_hubert/avhubert"

os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 5000

ngrok.set_auth_token("2LApBWQWx5ep57uYtAZ1egBcQl5_82cYLVbfPHabeWrTbGRcr")
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url,port))

app.config["BASE_URL"] = public_url
app.config["UPLOAD_FOLDER"] = '/content/data/'

@app.route("/")
def home():
  return "Hi"

@app.route("/aakash")
def aakash():
  return "Aakash"


@app.route("/upload", methods=['POST'])
def upload_video():
  if 'file' not in request.files:
    return jsonify({'error' : 'Media not provided'}), 400
  file = request.files['file']
  if file.filename == '':
    return jsonify({'error': 'No file selected'}), 400
  if file:
    filename = secure_filename(file.filename)
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], "clip.mp4"))
    preprocess_video(origin_clip_path, mouth_roi_path, face_predictor_path, mean_face_path)
    hypo = predict(mouth_roi_path, ckpt_path, user_dir)
    emotion = getEmotion()
  return jsonify({'msg' : hypo,'emotion' : emotion})


@app.route("/upload_hindi", methods=['POST'])
def upload_video_hindi():
  if 'file' not in request.files:
    return jsonify({'error' : 'Media not provided'}), 901
  file = request.files['file']
  if file.filename == '':
    return jsonify({'error': 'No file selected'}), 900
  if file:
    filename = secure_filename(file.filename)
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], "clip.mp4"))
    preprocess_video(origin_clip_path, mouth_roi_path, face_predictor_path, mean_face_path)
    hypo = predict(mouth_roi_path, ckpt_path_hindi, user_dir)
    print(hypo)
    emotion = getEmotion()
    print(emotion)
  return jsonify({'msg' : hypo,'emotion' : emotion})

threading.Thread(target=app.run,kwargs={"use_reloader": False}).start()


/content/av_hubert/avhubert


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * ngrok tunnel "https://affe-34-147-52-157.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
